#Instalação de Pacotes

In [1]:
!pip install -q yfinance

In [2]:
#Cotações do Yahoo Finance
import yfinance as yf

# Tratamento de Dados
import pandas as pd
from pandas.tseries.offsets import BDay
pd.set_option("display.max_colwidth", 150)

#Gráficos
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

#Utilidades
from datetime import date

## Funções

In [3]:
def busca_cadastro_cvm(data=(date.today()-BDay(2))):
  if data is not busca_cadastro_cvm.__defaults__[0]:
    data = pd.to_datetime(data)

  try:
    url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv'.format(data.year, data.month, data.day)
    return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

  except:
    print("Arquivo {} não encontrado!".format(url))
    print("Forneça outra data!")

In [4]:
def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq='MS')
  informe_completo = pd.DataFrame()

  for data in datas:
    try:
      url ='http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.zip'.format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=';')

    except:
      print("Arquivo {} não encontrado!".format(url))

    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)

  return informe_completo

In [5]:
def busca_informes_diarios_cvm_antes21(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim)
  informe_completoh = pd.DataFrame()

  for data in datas:
    try:
      url ='https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/HIST/inf_diario_fi_{}.zip'.format(data.year)
      informe_mensalh = pd.read_csv(url, sep=';')


    except:
      print("Arquivo {} não encontrado!".format(url))

    informe_completoh = pd.concat([informe_completoh, informe_mensalh], ignore_index=True)
  return informe_completoh

In [6]:
def melhores_e_piores(informes, cadastro, top=66, minimo_de_cotistas=33, classe=''):
  cadastro      = cadastro[cadastro['SIT'] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes['NR_COTST'] >= minimo_de_cotistas]
  cnpj_informes = fundos['CNPJ_FUNDO'].drop_duplicates()

  fundos = fundos.pivot_table(index='DT_COMPTC', columns='CNPJ_FUNDO')
  cotas_normalizadas = fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0]

  if classe == 'multimercado':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Multimercado']['CNPJ_FUNDO']
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'acoes':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Ações']['CNPJ_FUNDO']
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'rendafixa':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo de Renda Fixa']['CNPJ_FUNDO']
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  if classe == 'cambial':
    cnpj_cadastro      = cadastro[cadastro['CLASSE'] == 'Fundo Cambial']['CNPJ_FUNDO']
    cotas_normalizadas = cotas_normalizadas[cnpj_cadastro[cnpj_cadastro.isin(cnpj_informes)]]

  #melhores
  melhores = pd.DataFrame()
  melhores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=False)[:top] - 1) * 100
  for cnpj in melhores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    melhores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    melhores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    melhores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]

  #piores
  piores = pd.DataFrame()
  piores['retorno(%)'] = (cotas_normalizadas.iloc[-1].sort_values(ascending=True)[:top] - 1) * 100
  for cnpj in piores.index:
    fundo = cadastro[cadastro['CNPJ_FUNDO'] == cnpj]
    piores.at[cnpj, 'Fundo de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
    piores.at[cnpj, 'Classe']                = fundo['CLASSE'].values[0]
    piores.at[cnpj, 'PL']                    = fundo['VL_PATRIM_LIQ'].values[0]

  return melhores, piores

In [7]:
def consulta_fundo(informes, cnpj):
  fundo = informes[informes['CNPJ_FUNDO'] == cnpj].copy()
  fundo.set_index('DT_COMPTC', inplace=True)
  fundo['cotas_normalizadas'] = (fundo['VL_QUOTA'] / fundo['VL_QUOTA'].iloc[0])*100
  return fundo

In [8]:
def cdi_acumulado(data_inicio, data_fim):
  codigo_bcb = 12

  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  cdi = pd.read_json(url)
  cdi['data'] = pd.to_datetime(cdi['data'], dayfirst=True)
  cdi.set_index('data', inplace=True)

  cdi_acumulado = (1 + cdi[data_inicio : data_fim] / 100).cumprod()
  cdi_acumulado.iloc[0] = 1
  return cdi_acumulado

In [9]:
def selic_acumulada(data_inicio, data_fim):
  codigo_bcb = 4390

  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  selic = pd.read_json(url)
  selic['data'] = pd.to_datetime(selic['data'], dayfirst=True)
  selic.set_index('data', inplace=True)

  selic_acumulada = (1 + selic[data_inicio : data_fim] / 100).cumprod()
  selic_acumulada.iloc[0] = 1
  return selic_acumulada

## Benckmarks

In [10]:
cdi = cdi_acumulado('2021-01-01', '2022-03-04') * 100
cdi

,valor
data,
2021-01-04,100.000000
2021-01-05,100.014939
2021-01-06,100.022409
2021-01-07,100.029879
2021-01-08,100.037351
...,...
2022-02-24,105.939944
2022-02-25,105.982498
2022-03-02,106.025069


In [11]:
selic= selic_acumulada('2021-01-01', '2022-03-04') * 100
selic

,valor
data,
2021-01-01,100.000000
2021-02-01,100.280195
2021-03-01,100.480755
2021-04-01,100.691765
2021-05-01,100.963633
2021-06-01,101.276620
2021-07-01,101.641216
2021-08-01,102.078273
2021-09-01,102.527417


In [12]:
ibov = yf.download('^BVSP', start='2021-01-01', end='2022-03-04')['Close']
ibov = (ibov / ibov.iloc[0])*100
ibov

[*********************100%%**********************]  1 of 1 completed


Date
2021-01-04    100.000000
2021-01-05    100.560907
2021-01-06    101.090605
2021-01-07    102.866108
2021-01-08    105.498575
                 ...    
2022-02-23     94.475278
2022-02-24     94.124395
2022-02-25     95.431772
2022-03-02     97.145701
2022-03-03     97.138953
Name: Close, Length: 289, dtype: float64

## Buscando Dados da CVM

In [13]:
cadastro = busca_cadastro_cvm()
cadastro

<ipython-input-3-ed827a3a231c>:7: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(url, sep=';', encoding='ISO-8859-1')


,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,CD_CVM,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,...,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR,INVEST_CEMPR_EXTER,CLASSE_ANBIMA
0,FACFIF,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCEIRO - MAX,2003-04-30,1994-12-20,19.0,2000-08-01,CANCELADA,2000-08-01,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FACFIF,00.000.731/0001-37,ITAMARITI CASH FUNDO APLICACAO QUOTAS FDOS INVESTIMENTO,2003-04-30,1994-05-18,40681.0,1996-01-26,CANCELADA,1996-01-26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FACFIF,00.000.732/0001-81,FUNDO APLIC. QUOTAS DE F.I. SANTANDER CURTO PRAZO,2003-04-30,1994-05-24,27.0,1999-09-03,CANCELADA,1999-09-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FACFIF,00.000.740/0001-28,FUNDO DE APLIC EM QUOTAS DE FUNDOS DE INV BMC 60 DIAS URV,2003-04-30,1994-05-23,40690.0,1996-06-10,CANCELADA,1996-06-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FACFIF,00.000.749/0001-39,BALANCE FUNDO APLICACAO QUOTAS FUNDO INVESTIMENTO FINANCEIRO,2003-04-30,1994-05-12,35.0,2000-06-26,CANCELADA,2000-06-26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79033,FUNCINE,31.674.280/0001-47,MEDIA VENTURE ESG INCENTIVADO - FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGRAFICA NACIONAL,2018-10-26,2018-09-13,418001.0,2023-07-21,CANCELADA,2023-07-21,NaN,...,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.,16.549.480/0001-84,RSM BRASIL AUDITORES INDEPENDENTES LTDA.,16.695.922/0001-09,ID CORRETORA DE TITULOS E VALORES MOBILIARIOS S.A.,NaN,NaN,NaN,NaN
79034,FUNCINE,32.222.962/0001-81,Fundo de Financiamento da Indústria Cinematográfica Nacional de Minas Gerais,2018-12-20,2018-11-14,418002.0,2021-10-29,CANCELADA,2021-10-29,NaN,...,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.,15.489.568/0001-95,INTRA INVESTIMENTOS DTVM LTDA,NaN,NaN,NaN,NaN
79035,FUNCINE,48.879.897/0001-00,FUNDO DE FINANCIAMENTO DA INDÚSTRIA CINEMATOGRÁFICA NACIONAL - FUNCINE CEDRO LIGOCKI,2022-12-13,2022-12-01,422001.0,NaN,FASE PRÉ-OPERACIONAL,2022-12-13,2022-12-13,...,18.190.296/0001-16,CEDRO ASSET MANAGEMENT LTDA,19.280.834/0001-26,NEXT AUDITORES INDEPENDENTES S/S LTDA.,16.695.922/0001-09,ID CORRETORA DE TITULOS E VALORES MOBILIARIOS S.A.,NaN,NaN,NaN,NaN
79036,FUNCINE,52.714.528/0001-26,INVESTIMAGE 4 - FUNDO DE FINANCIAMENTO DA INDÚSTRIA CINEMATOGRÁFICA,2023-10-30,2023-10-17,423001.0,NaN,EM FUNCIONAMENTO NORMAL,2023-12-28,2023-12-28,...,09.544.103/0001-97,INVESTIMAGE ADMINISTRADORA DE RECURSOS LTDA.,19.280.834/0001-26,NEXT AUDITORES INDEPENDENTES S/S LTDA.,16.695.922/0001-09,ID CORRETORA DE TITULOS E VALORES MOBILIARIOS S.A.,NaN,NaN,NaN,NaN


In [14]:
#informe_hi = busca_informes_diarios_cvm_antes21(data_inicio='2013', data_fim='2020')
#informe_hi


In [15]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2021-01', data_fim='2022-03')
informes

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2021-01-04,1097664.87,27.503336,1095773.57,0.0,0.0,1
1,FI,00.017.024/0001-53,2021-01-05,1097742.61,27.503455,1095778.31,0.0,0.0,1
2,FI,00.017.024/0001-53,2021-01-06,1097837.04,27.503196,1095768.02,0.0,0.0,1
3,FI,00.017.024/0001-53,2021-01-07,1097939.54,27.503352,1095774.20,0.0,0.0,1
4,FI,00.017.024/0001-53,2021-01-08,1096790.38,27.503710,1095788.49,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
6736163,FI,97.929.213/0001-34,2022-03-25,72973880.29,9.601337,72963645.88,0.0,0.0,2
6736164,FI,97.929.213/0001-34,2022-03-28,73011911.38,9.606289,73001275.99,0.0,0.0,2
6736165,FI,97.929.213/0001-34,2022-03-29,72973008.18,9.604625,72988633.14,0.0,0.0,2
6736166,FI,97.929.213/0001-34,2022-03-30,72867250.91,9.587158,72855897.94,0.0,0.0,2


#Filtro FundoTCE

In [16]:
austro= informes.loc[informes['CNPJ_FUNDO']=='18.366.002/0001-64']

In [17]:
austro

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
140354,FI,18.366.002/0001-64,2021-01-04,86206440.26,1.373842,80468738.42,0.0,0.00,33
140355,FI,18.366.002/0001-64,2021-01-05,86200504.38,1.373650,80457480.95,0.0,0.00,33
140356,FI,18.366.002/0001-64,2021-01-06,86194568.85,1.373458,80446224.55,0.0,0.00,33
140357,FI,18.366.002/0001-64,2021-01-07,86188633.70,1.373266,80434969.25,0.0,0.00,33
140358,FI,18.366.002/0001-64,2021-01-08,86182698.91,1.373073,80423715.04,0.0,0.00,33
...,...,...,...,...,...,...,...,...,...
5916457,FI,18.366.002/0001-64,2022-02-25,15473390.59,0.970290,8555747.25,0.0,0.00,33
6373370,FI,18.366.002/0001-64,2022-03-02,15473390.59,0.970195,8554910.72,0.0,0.00,33
6373371,FI,18.366.002/0001-64,2022-03-03,15473390.59,0.970100,8554074.24,0.0,0.00,33
6373372,FI,18.366.002/0001-64,2022-03-04,0.00,0.970005,0.00,0.0,8553237.87,0


In [18]:
#Transformar o index em data

In [19]:
#austro.index = pd.to_datetime(austro.index, format="%d/%m/%Y")

In [20]:
austro['cotan']= austro['VL_QUOTA']/austro['VL_QUOTA'].iloc[0]*100

<ipython-input-20-8c7e76458553>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  austro['cotan']= austro['VL_QUOTA']/austro['VL_QUOTA'].iloc[0]*100


In [21]:
austro


,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST,cotan
140354,FI,18.366.002/0001-64,2021-01-04,86206440.26,1.373842,80468738.42,0.0,0.00,33,100.000000
140355,FI,18.366.002/0001-64,2021-01-05,86200504.38,1.373650,80457480.95,0.0,0.00,33,99.986010
140356,FI,18.366.002/0001-64,2021-01-06,86194568.85,1.373458,80446224.55,0.0,0.00,33,99.972022
140357,FI,18.366.002/0001-64,2021-01-07,86188633.70,1.373266,80434969.25,0.0,0.00,33,99.958034
140358,FI,18.366.002/0001-64,2021-01-08,86182698.91,1.373073,80423715.04,0.0,0.00,33,99.944049
...,...,...,...,...,...,...,...,...,...,...
5916457,FI,18.366.002/0001-64,2022-02-25,15473390.59,0.970290,8555747.25,0.0,0.00,33,70.626010
6373370,FI,18.366.002/0001-64,2022-03-02,15473390.59,0.970195,8554910.72,0.0,0.00,33,70.619104
6373371,FI,18.366.002/0001-64,2022-03-03,15473390.59,0.970100,8554074.24,0.0,0.00,33,70.612199
6373372,FI,18.366.002/0001-64,2022-03-04,0.00,0.970005,0.00,0.0,8553237.87,0,70.605295


#FUNDOS MULTIMERCADO

In [22]:
melhores, piores = melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe='multimercado')


TypeError: Could not convert FI to numeric

In [ ]:
melhores

#exporta para excell

In [ ]:
piores.to_excel(r'C:\Users\Felipe\Desktop\export_dataframe.xlsx', index=False)

#Gráfico

In [ ]:
cotas = pd.DataFrame()
fundos_destaque = melhores
for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']

In [ ]:
cotas

In [ ]:
fig = go.Figure()

traces = [go.Scatter(
    x = cotas.index,
    y = cotas[fundo],
    name = fundo
) for fundo in cotas.columns]

fig.add_trace(go.Scatter(x=austro['DT_COMPTC'], y=austro['cotan'], name='Austro', line=dict(color='black', width=3)))

fig.add_traces(traces)
fig.update_layout(title='Melhores Fundos multimercado')
fig.update_layout(legend_orientation="h")

fig.show()

In [ ]:
cotas = pd.DataFrame()
fundos_destaque = piores
for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']

In [ ]:
fig = go.Figure()

traces = [go.Scatter(
    x = cotas.index,
    y = cotas[fundo],
    name = fundo
) for fundo in cotas.columns]

fig.add_trace(go.Scatter(x=ibov.index, y=ibov, name='IBOV', line=dict(color='black', width=3)))

fig.add_traces(traces)
fig.update_layout(title='Piores Fundos multimercado')
fig.update_layout(legend_orientation="h")

fig.show()

## Fundos de Ações

In [ ]:
melhores, piores = melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe='acoes')

In [ ]:
melhores

In [ ]:
piores

### Graficos

In [ ]:
cotas = pd.DataFrame()
fundos_destaque = pd.concat([melhores, piores])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']

In [ ]:
cotas

In [ ]:
fig = go.Figure()

traces = [go.Scatter(
    x = cotas.index,
    y = cotas[fundo],
    name = fundo
) for fundo in cotas.columns]

fig.add_trace(go.Scatter(x=ibov.index, y=ibov, name='IBOV', line=dict(color='black', width=3)))

fig.add_traces(traces)
fig.update_layout(title='Melhores e Piores Fundos de Ações do 1º Semestre de 2022')
fig.update_layout(legend_orientation="h")

fig.show()

## Fundos Cambial


In [ ]:
melhores, piores = melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe='cambial')

In [ ]:
melhores

In [ ]:
piores

### Grafico de Cotas

In [ ]:
cotas = pd.DataFrame()
fundos_destaque =  pd.concat([melhores, piores])

for cnpj in fundos_destaque.index:
  cotas = pd.concat([cotas, consulta_fundo(informes, cnpj)['cotas_normalizadas']], axis=1)

cotas.columns = fundos_destaque['Fundo de Investimento']

In [ ]:
cotas

In [ ]:
cotas

In [ ]:
ibov

In [ ]:
fig = go.Figure()

traces = [go.Scatter(
    x = cotas.index,
    y = cotas[fundo],
    name = fundo
) for fundo in cotas.columns]

fig.add_trace(go.Scatter(x=ibov.index, y=ibov, name='IBOV', line=dict(color='black', width=3)))

fig.add_traces(traces)
fig.update_layout(title='Melhores Fundos Cambiais do 1º Semestre de 2022')
fig.update_layout(legend_orientation="h")

fig.show()

In [ ]:
cotas.columns

### Correlação

In [ ]:
sns.heatmap(cotas.iloc[:, :5].corr(), annot=True);

In [ ]:
sns.heatmap(cotas.iloc[:, 6:].corr(), annot=True);